# Data Processing using Pyspark

In [1]:
#%pip install pyarrow

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1709849208178_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Transient to raw

### load source 1 climatedata_imf_org

In [3]:
# Load csv Dataset
df_source_1=spark.read.csv('s3://trabajo1datalake/transient/climatedata_imf_org/level_sea/Change_in_Mean_Sea_Levels.csv',
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#shape of dataset
print((df_source_1.count(),len(df_source_1.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(35604, 13)

In [5]:
#load data into raw
pathparquet_out='s3://trabajo1datalake/raw/climatedata_imf_org/level_sea'
#save the data into parquet format
df_source_1.write.mode('overwrite').format('parquet').save(pathparquet_out)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Load csv Dataset
df_source_1_1=spark.read.csv('s3://trabajo1datalake/transient/climatedata_imf_org/dioxido_carbono/Atmospheric_CO2_Concentrations.csv',
                  inferSchema=True,
                  header=True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#shape of dataset
print((df_source_1_1.count(),len(df_source_1_1.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1570, 12)

In [8]:
#load data into raw
pathparquet_out='s3://trabajo1datalake/raw/climatedata_imf_org/dioxido_carbono'
#save the data into parquet format
df_source_1_1.write.mode('overwrite').format('parquet').save(pathparquet_out)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### lod source 2 ourworldindata_org

In [9]:
# desde S3
df_source_2=spark.read.csv('s3://trabajo1datalake/transient/ourworldindata_org/owid-co2-data/owid-co2-data.csv',
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#shape of dataset
print((df_source_2.count(),len(df_source_1.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(48058, 13)

In [11]:
#load data into raw
pathparquet_out_2='s3://trabajo1datalake/raw/ourworldindata_org/owid-co2-data'
#save the data into parquet format
df_source_2.write.mode('overwrite').format('parquet').save(pathparquet_out_2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Raw to trusted

Limpiar los datos: 
* identificar , eliminar o completar nulos
* eliminar duplicados
* cambiar formatos de datos
* corregir valores y/o strings

### Clean data source 1

In [12]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import StringType,DoubleType,IntegerType
from pyspark.sql.functions import isnan, when, count, col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
path_raw_source_1='s3://trabajo1datalake/raw/climatedata_imf_org/level_sea'
# Load raw data
df_raw_source_1=spark.read.parquet(path_raw_source_1,
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#printSchema
df_raw_source_1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Value: double (nullable = true)

In [15]:
#columns of dataframe
df_raw_source_1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'Measure', 'Date', 'Value']

In [16]:
#shape of dataset
print((df_raw_source_1.count(),len(df_raw_source_1.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(35604, 13)

In [17]:
def identificar_nulos(df):
    #identificar nulos
    df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def eliminar_rellenar_nulos(df,tarea='eliminar', dict_rellenar = {'columna': 0}): 
    # Eliminar filas con valores nulos
    if tarea == 'eliminar':
        df_clean = df.dropna()
    elif tarea == 'completar':
        # Rellenar valores nulos con un valor específico
        df_clean = df.fillna(dict_rellenar) # dict_rellenar = {'columna': valor}
    else:
        df_clean = df
        print("tarea no valida")
    
    return df_clean

def eliminar_duplicados(df):
    # Eliminar duplicados
    df_clean = df.dropDuplicates()
    return df_clean

def corregir_valores(columna,condicion,nuevo_valor):
    df_clean = df.withColumn(f'{columna}', when(condicion, nuevo_valor).otherwise(df[f'{columna}']))
    return df_clean

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#fisrt few rows of dataframe
df_raw_source_1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+--------------+-----------+------+
|ObjectId|Country|ISO2|ISO3|           Indicator|       Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|       Measure|       Date| Value|
+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+--------------+-----------+------+
|       1|  World|null| WLD|Change in mean se...|Millimeters|National Oceanic ...|    ECCL|Change in Mean Se...|Environment, Clim...|   Andaman Sea|D12/17/1992|-10.34|
|       2|  World|null| WLD|Change in mean se...|Millimeters|National Oceanic ...|    ECCL|Change in Mean Se...|Environment, Clim...|   Arabian Sea|D12/17/1992|-18.46|
|       3|  World|null| WLD|Change in mean se...|Millimeters|National Oceanic ...|    ECCL|Change in Mean Se...|Environment, Clim...|Atlantic Ocean|D12/17/1992|

In [20]:
print(df_raw_source_1.count())
# Eliminar duplicados
df_trusted_source_1 = eliminar_duplicados(df_raw_source_1)
print(df_trusted_source_1.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

35604
35604

In [21]:
#identificar nulos
identificar_nulos(df_trusted_source_1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+-----+----+---------+----+------+--------+--------+-------------------+-------+----+-----+
|ObjectId|Country| ISO2|ISO3|Indicator|Unit|Source|CTS_Code|CTS_Name|CTS_Full_Descriptor|Measure|Date|Value|
+--------+-------+-----+----+---------+----+------+--------+--------+-------------------+-------+----+-----+
|       0|      0|35604|   0|        0|   0|     0|       0|       0|                  0|      0|   0|    0|
+--------+-------+-----+----+---------+----+------+--------+--------+-------------------+-------+----+-----+

In [22]:
# eliminar rellenar nulos
df_trusted_source_1 = eliminar_rellenar_nulos(df_trusted_source_1,tarea='completar', dict_rellenar = {'ISO2': 'NA'})
#identificar nulos
identificar_nulos(df_trusted_source_1)
print(df_trusted_source_1.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+---------+----+------+--------+--------+-------------------+-------+----+-----+
|ObjectId|Country|ISO2|ISO3|Indicator|Unit|Source|CTS_Code|CTS_Name|CTS_Full_Descriptor|Measure|Date|Value|
+--------+-------+----+----+---------+----+------+--------+--------+-------------------+-------+----+-----+
|       0|      0|   0|   0|        0|   0|     0|       0|       0|                  0|      0|   0|    0|
+--------+-------+----+----+---------+----+------+--------+--------+-------------------+-------+----+-----+

35604

In [23]:
# casting columnas de ObjectID y fecha
from pyspark.sql.functions import col

#pasar datos de enteros a string
df_trusted_source_1 = df_trusted_source_1.withColumn('ObjectId', col('ObjectId').cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_trusted_source_1.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+------------+-----------+-----+
|ObjectId|Country|ISO2|ISO3|           Indicator|       Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|     Measure|       Date|Value|
+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+------------+-----------+-----+
|    1250|  World|  NA| WLD|Change in mean se...|Millimeters|National Oceanic ...|    ECCL|Change in Mean Se...|Environment, Clim...|Adriatic Sea|D05/27/1994|17.79|
+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+------------+-----------+-----+
only showing top 1 row

In [25]:
from pyspark.sql.functions import to_date
#pasar datos de string a fecha
df_trusted_source_1   = df_trusted_source_1.withColumn("Date", regexp_replace("Date", "D", "")).withColumn("Date", to_date("Date", "MM/dd/yyyy"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#printSchema
df_trusted_source_1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = false)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: double (nullable = true)

In [27]:
# Corregir valores incorrectos
#corregir_valores(columna,condición,nuevo_valor)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_trusted_source_1.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+------------+----------+-----+
|ObjectId|Country|ISO2|ISO3|           Indicator|       Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|     Measure|      Date|Value|
+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+------------+----------+-----+
|    1250|  World|  NA| WLD|Change in mean se...|Millimeters|National Oceanic ...|    ECCL|Change in Mean Se...|Environment, Clim...|Adriatic Sea|1994-05-27|17.79|
+--------+-------+----+----+--------------------+-----------+--------------------+--------+--------------------+--------------------+------------+----------+-----+
only showing top 1 row

In [29]:

def corregir_nombres_columnas_string(df):
    # Limpiar espacios y caracteres especiales en todas las columnas de texto
    for column in df.columns:
        if df.schema[column].dataType is StringType() :
            print(column)
            df = df.withColumn(column, regexp_replace(col(column), "[^a-zA-Z0-9\s]+", "").alias(column)).withColumn(column, regexp_replace(col(column), "\s+", "_").alias(column))
    return df


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Seleccionar las columnas de texto del DataFrame
text_columns = [column for column, data_type in df_trusted_source_1.dtypes if data_type == "string"]
df_trusted_source_1.select('Indicator').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           Indicator|
+--------------------+
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
|Change in mean se...|
+--------------------+
only showing top 10 rows

In [31]:
#corregir textos de columnas string
df_trusted_source_1 = corregir_nombres_columnas_string(df_trusted_source_1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ObjectId
Country
ISO2
ISO3
Indicator
Unit
Source
CTS_Code
CTS_Name
CTS_Full_Descriptor
Measure

In [32]:
df_trusted_source_1.select('Indicator').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           Indicator|
+--------------------+
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
|Change_in_mean_se...|
+--------------------+
only showing top 10 rows

In [33]:
#guardar dataset limpio
path_trusted_source_1='s3://trabajo1datalake/trusted/climatedata_imf_org/level_sea'
#save the data into parquet format
df_trusted_source_1.write.mode('overwrite').format('parquet').save(path_trusted_source_1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Clean data source 1.1

In [36]:
path_raw_source_1_1='s3://trabajo1datalake/raw/climatedata_imf_org/dioxido_carbono'
# Load raw data
df_raw_source_1_1=spark.read.parquet(path_raw_source_1_1,
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#printSchema
df_raw_source_1_1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Value: double (nullable = true)

In [39]:
#fisrt few rows of dataframe
df_raw_source_1_1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+-------+------+
|ObjectId|Country|ISO2|ISO3|           Indicator|             Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|   Date| Value|
+--------+-------+----+----+--------------------+-----------------+--------------------+--------+--------------------+--------------------+-------+------+
|       1|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1958M03| 315.7|
|       2|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1958M04|317.45|
|       3|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr. Pieter Tans, ...|    ECCA|Atmospheric Carbo...|Environment, Clim...|1958M05|317.51|
|       4|  World|null| WLD|Monthly Atmospher...|Parts Per Million|Dr.

In [54]:
from pyspark.sql.functions import udf

# Definir una función UDF para convertir la cadena a formato de fecha
def convert_to_date(date_string):
    year = date_string[:4]
    month = date_string[5:]
    formatted_date = f"{year}/{month}"
    return formatted_date

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
print(df_raw_source_1_1.count())
# Eliminar duplicados
df_trusted_source_1_1 = eliminar_duplicados(df_raw_source_1_1)
print(df_trusted_source_1_1.count())

# eliminar rellenar nulos
df_trusted_source_1_1 = eliminar_rellenar_nulos(df_trusted_source_1_1,tarea='completar', dict_rellenar = {'ISO2': 'NA'})
#identificar nulos
identificar_nulos(df_trusted_source_1_1)
print(df_trusted_source_1_1.count())

#pasar datos de enteros a string
df_trusted_source_1_1 = df_trusted_source_1_1.withColumn('ObjectId', col('ObjectId').cast('string'))

from pyspark.sql.functions import to_date
#pasar datos de string a fecha

convert_to_date_udf = udf(convert_to_date, StringType())

# Aplicar la función definida a la columna y convertirla al tipo de datos DateType
df_trusted_source_1_1 = df_trusted_source_1_1.withColumn("Date", to_date(convert_to_date_udf(col("Date")), "yyyy/MM"))


#corregir textos de columnas string
df_trusted_source_1_1 = corregir_nombres_columnas_string(df_trusted_source_1_1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1570
1570
+--------+-------+----+----+---------+----+------+--------+--------+-------------------+----+-----+
|ObjectId|Country|ISO2|ISO3|Indicator|Unit|Source|CTS_Code|CTS_Name|CTS_Full_Descriptor|Date|Value|
+--------+-------+----+----+---------+----+------+--------+--------+-------------------+----+-----+
|       0|      0|   0|   0|        0|   0|     0|       0|       0|                  0|   0|    0|
+--------+-------+----+----+---------+----+------+--------+--------+-------------------+----+-----+

1570
ObjectId
Country
ISO2
ISO3
Indicator
Unit
Source
CTS_Code
CTS_Name
CTS_Full_Descriptor

In [57]:
#printSchema
df_trusted_source_1_1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = false)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: double (nullable = true)

In [58]:
df_trusted_source_1_1.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+----------+-----+
|ObjectId|Country|ISO2|ISO3|           Indicator|   Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|      Date|Value|
+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+----------+-----+
|     268|  World|  NA| WLD|Monthly_Atmospher...|Percent|Dr_Pieter_Tans_Na...|    ECCA|Atmospheric_Carbo...|Environment_Clima...|1969-10-01| 0.48|
+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 1 row

In [59]:
#guardar dataset limpio
path_trusted_source_1_1='s3://trabajo1datalake/trusted/climatedata_imf_org/dioxido_carbono'
#save the data into parquet format
df_trusted_source_1_1.write.mode('overwrite').format('parquet').save(path_trusted_source_1_1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Clean data source 2

In [60]:
path_raw_source_2='s3://trabajo1datalake/raw/ourworldindata_org/owid-co2-data'
# Load raw data
df_raw_source_2=spark.read.parquet(path_raw_source_2,
                  inferSchema=True,
                  header=True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
#printSchema
df_raw_source_2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- iso_code: string (nullable = true)
 |-- population: long (nullable = true)
 |-- gdp: double (nullable = true)
 |-- cement_co2: double (nullable = true)
 |-- cement_co2_per_capita: double (nullable = true)
 |-- co2: double (nullable = true)
 |-- co2_growth_abs: double (nullable = true)
 |-- co2_growth_prct: double (nullable = true)
 |-- co2_including_luc: double (nullable = true)
 |-- co2_including_luc_growth_abs: double (nullable = true)
 |-- co2_including_luc_growth_prct: double (nullable = true)
 |-- co2_including_luc_per_capita: double (nullable = true)
 |-- co2_including_luc_per_gdp: double (nullable = true)
 |-- co2_including_luc_per_unit_energy: double (nullable = true)
 |-- co2_per_capita: double (nullable = true)
 |-- co2_per_gdp: double (nullable = true)
 |-- co2_per_unit_energy: double (nullable = true)
 |-- coal_co2: double (nullable = true)
 |-- coal_co2_per_capita: double (nullable = true

In [62]:
#shape of dataset
print((df_raw_source_2.count(),len(df_raw_source_2.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(48058, 79)

In [63]:
#fisrt few rows of dataframe
df_raw_source_2.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+--------+----------+----+----------+---------------------+----+--------------+---------------+-----------------+----------------------------+-----------------------------+----------------------------+-------------------------+---------------------------------+--------------+-----------+-------------------+--------+-------------------+---------------+--------------------------+-----------------------+---------------------+--------------+----------------------------+-------------------+----------------------+------------------+------------------+------------------+--------------------+-----------------+--------------+-----------+----------------------+-------+------------------+-----------------------------+--------------+-------------------+------------------------------+-------+------------------+-------------+------------------------+-------+------------------+--------------------+------------------+--------------------------+-----------------------+----------------+

In [64]:
print(df_raw_source_2.count())
# Eliminar duplicados
df_trusted_source_2 = eliminar_duplicados(df_raw_source_2)
print(df_trusted_source_2.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48058
48058

In [65]:
#identificar nulos
identificar_nulos(df_trusted_source_2.select('country','year','population','co2','co2_growth_abs','co2_growth_prct','co2_per_capita','temperature_change_from_co2'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+----------+-----+--------------+---------------+--------------+---------------------------+
|country|year|population|  co2|co2_growth_abs|co2_growth_prct|co2_per_capita|temperature_change_from_co2|
+-------+----+----------+-----+--------------+---------------+--------------+---------------------------+
|      0|   0|      8563|17750|         19901|          22922|         21458|                       6334|
+-------+----+----------+-----+--------------+---------------+--------------+---------------------------+

In [66]:
from pyspark.sql.functions import when
from pyspark.sql.types import StringType,DoubleType,IntegerType, LongType

# Reemplazar valores nulos en columnas
def remplazar_nulos_por_tipo(df):
    for column in df.columns:
        if df.schema[column].dataType is StringType():
            df = df.withColumn(column, when(df[column].isNull(), "NA").otherwise(df[column]))
        elif df.schema[column].dataType is LongType():
            df = df.withColumn(column, when(df[column].isNull(), 0).otherwise(df[column]))
        elif df.schema[column].dataType is IntegerType():
            df = df.withColumn(column, when(df[column].isNull(), 0).otherwise(df[column]))
        elif df.schema[column].dataType is DoubleType():
            df = df.withColumn(column, when(df[column].isNull(), 0.0).otherwise(df[column]))
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# eliminar rellenar nulos
df_trusted_source_2 = remplazar_nulos_por_tipo(df_trusted_source_2)
#identificar nulos
#identificar nulos
identificar_nulos(df_trusted_source_2.select('country','year','population','co2','co2_growth_abs','co2_growth_prct','co2_per_capita','temperature_change_from_co2'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+----------+---+--------------+---------------+--------------+---------------------------+
|country|year|population|co2|co2_growth_abs|co2_growth_prct|co2_per_capita|temperature_change_from_co2|
+-------+----+----------+---+--------------+---------------+--------------+---------------------------+
|      0|   0|         0|  0|             0|              0|             0|                          0|
+-------+----+----------+---+--------------+---------------+--------------+---------------------------+

In [68]:
df_trusted_source_2.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+--------+----------+---------------+----------+---------------------+-------+--------------+---------------+-----------------+----------------------------+-----------------------------+----------------------------+-------------------------+---------------------------------+--------------+-----------+-------------------+--------+-------------------+---------------+--------------------------+-----------------------+---------------------+--------------+----------------------------+-------------------+----------------------+------------------+------------------+------------------+--------------------+-----------------+--------------+-----------+----------------------+-------+------------------+-----------------------------+--------------+-------------------+------------------------------+-------+------------------+-------------+------------------------+-------+------------------+--------------------+------------------+--------------------------+-----------------------+-------

In [69]:
#guardar dataset limpio
path_trusted_source_2='s3://trabajo1datalake/trusted/ourworldindata_org/owid-co2-data'
#save the data into parquet format
df_trusted_source_2.write.mode('overwrite').format('parquet').save(path_trusted_source_2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…